# Imports 

This notebook is prepared for selecting random sample of prompts from human eval dataset based on their implementation difficulties determined by an LLM. 

In [2]:
import pandas as pd 
import numpy as np

# Load data

In [3]:
df = pd.read_csv('../data.csv') # human eval data in csv format
df.head()

,task_id,prompt,entry_point,canonical_solution,test
0,HumanEval/0,from typing import List\n\n\ndef has_close_ele...,has_close_elements,"for idx, elem in enumerate(numbers):\n ...","\n\nMETADATA = {\n 'author': 'jt',\n 'da..."
1,HumanEval/1,from typing import List\n\n\ndef separate_pare...,separate_paren_groups,result = []\n current_string = []\n ...,"\n\nMETADATA = {\n 'author': 'jt',\n 'da..."
2,HumanEval/2,\n\ndef truncate_number(number: float) -> floa...,truncate_number,return number % 1.0\n,"\n\nMETADATA = {\n 'author': 'jt',\n 'da..."
3,HumanEval/3,from typing import List\n\n\ndef below_zero(op...,below_zero,balance = 0\n\n for op in operations:\n...,"\n\nMETADATA = {\n 'author': 'jt',\n 'da..."
4,HumanEval/4,from typing import List\n\n\ndef mean_absolute...,mean_absolute_deviation,mean = sum(numbers) / len(numbers)\n re...,"\n\nMETADATA = {\n 'author': 'jt',\n 'da..."


In [4]:
print(df.iloc[0]['prompt'])


from typing import List


def has_close_elements(numbers: List[float], threshold: float) -> bool:
    """ Check if in given list of numbers, are any two numbers closer to each other than
    given threshold.
    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)
    False
    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)
    True
    """



# Preprocess data

In [6]:
functions = pd.read_csv('../functions.csv')# functions are grouped by their diff level of impl.
functions.head()

,entry_point,difficulty_level
0,has_close_elements,medium
1,separate_paren_groups,medium
2,truncate_number,medium
3,below_zero,medium
4,mean_absolute_deviation,easy


In [7]:
fun_easy_to_code = functions[functions['difficulty_level'] == 'easy'].drop('difficulty_level', axis=1).to_numpy(dtype=str).flatten()
fun_medium_to_code = functions[functions['difficulty_level'] == 'medium'].drop('difficulty_level', axis=1).to_numpy(dtype=str).flatten()
fun_hard_to_code = functions[functions['difficulty_level'] == 'hard'].drop('difficulty_level', axis=1).to_numpy(dtype=str).flatten()

### Shuffle the arrays and get 10 random samples

In [8]:
np.random.seed(42)

In [9]:
fun_samples_easy = np.random.permutation(fun_easy_to_code)[:10]
fun_samples_medium = np.random.permutation(fun_medium_to_code)[:10]
fun_samples_hard = np.random.permutation(fun_hard_to_code)[:10]


Now extract randomly selected 10 prompt from each category , 30 in total,
some of the entry points have same name so we better drop duplcates to ensure 10 prompts from each category

In [10]:
prompts_easy = df[df['entry_point'].isin(fun_samples_easy)].drop_duplicates(subset='entry_point')
prompts_medium = df[df['entry_point'].isin(fun_samples_medium)].drop_duplicates(subset='entry_point')
prompts_hard = df[df['entry_point'].isin(fun_samples_hard)].drop_duplicates(subset='entry_point')

get data frames as prompts and entry points only now

In [12]:
easy = prompts_easy[['entry_point', 'prompt']]
easy.to_csv('data_prompts/easy.csv', index=False)
medium = prompts_medium[['entry_point', 'prompt']]
medium.to_csv('data_prompts/medium.csv', index=False)
hard = prompts_hard[['entry_point', 'prompt']]
hard.to_csv('data_prompts/hard.csv', index=False)